# Exp with imperfection

In [34]:
from isingchat import ising
import numpy as np
import scipy

In [45]:
temp = 1
mag_field = 0
interactions=1/np.array([1,2])**2
interactions_2=1/np.array([1,2])**2
num_neighbors = 2

In [28]:
interactions_2

array([0.44444444, 0.16      ])

In [46]:
nnz_elems, nnz_rows, nnz_cols = ising._csr_log_transfer_matrix_parts_fast(
    temp, mag_field, interactions, num_neighbors
)

# Normalize nonzero matrix elements.
max_w_log_elem_1 = np.max(nnz_elems)
nnz_elems -= max_w_log_elem_1
norm_nnz_elems = np.exp(nnz_elems)
# Construct the sparse matrix.
num_rows = 2 ** num_neighbors
w_shape = (num_rows, num_rows)
w_matrix_1 = ising.csr_matrix(
    (norm_nnz_elems, (nnz_rows, nnz_cols)), shape=w_shape
)

print(w_matrix_1)


  (0, 0)	1.0
  (0, 1)	0.6065306597126334
  (1, 2)	0.1353352832366127
  (1, 3)	0.0820849986238988
  (2, 0)	0.0820849986238988
  (2, 1)	0.1353352832366127
  (3, 2)	0.6065306597126334
  (3, 3)	1.0


In [38]:
np.exp((-1/1.5**2+1/2.5**2))/np.exp(1/1.5**2+1/2.5**2)

0.41111229050718745

In [47]:
# Second matrix
nnz_elems, nnz_rows, nnz_cols = ising._csr_log_transfer_matrix_parts_fast(
    temp, mag_field, interactions_2, num_neighbors
)

# Normalize nonzero matrix elements.
max_w_log_elem_2 = np.max(nnz_elems)
nnz_elems -= max_w_log_elem_2
norm_nnz_elems = np.exp(nnz_elems)
# Construct the sparse matrix.
num_rows = 2 ** num_neighbors
w_shape = (num_rows, num_rows)
w_matrix_2 = ising.csr_matrix(
    (norm_nnz_elems, (nnz_rows, nnz_cols)), shape=w_shape
)
print(w_matrix_2)


  (0, 0)	1.0
  (0, 1)	0.6065306597126334
  (1, 2)	0.1353352832366127
  (1, 3)	0.0820849986238988
  (2, 0)	0.0820849986238988
  (2, 1)	0.1353352832366127
  (3, 2)	0.6065306597126334
  (3, 3)	1.0


In [64]:
w_matrix = w_matrix_1*w_matrix_2
print(w_matrix)

  (0, 3)	0.049787068367863944
  (0, 2)	0.0820849986238988
  (0, 1)	0.6065306597126334
  (0, 0)	1.0
  (1, 3)	0.0820849986238988
  (1, 2)	0.049787068367863944
  (1, 1)	0.018315638888734182
  (1, 0)	0.011108996538242308
  (2, 3)	0.011108996538242308
  (2, 2)	0.018315638888734182
  (2, 1)	0.049787068367863944
  (2, 0)	0.0820849986238988
  (3, 3)	1.0
  (3, 2)	0.6065306597126334
  (3, 1)	0.0820849986238988
  (3, 0)	0.049787068367863944


In [65]:
from scipy.sparse.linalg import eigs as sparse_eigs
# Evaluate the largest eigenvalue only.
num_eigvals = 1
w_norm_eigvals: np.ndarray
# noinspection PyTypeChecker
w_norm_eigvals, _ = sparse_eigs(
    w_matrix, k=num_eigvals, which="LM", return_eigenvectors=True
)
max_eigval = w_norm_eigvals.real[0]
# In the thermodynamic limit, the number of spins is infinity.
# Accordingly, only the largest reduced eigenvalue contributes.
cellunit = 2
helm_free_erg_tl = -(temp / cellunit) * (
    max_w_log_elem_1 + max_w_log_elem_2
    + np.log(max_eigval)
)

In [66]:
max_eigval

1.1113042731259941

In [53]:
max_w_log_elem_2

1.25

In [54]:
w_norm_eigvals_1, _ = sparse_eigs(
    w_matrix_1, k=num_eigvals, which="LM", return_eigenvectors=True
)
max_eigval_1 = w_norm_eigvals_1.real[0]

In [60]:
max_eigval_1**2

1.1113042731259937